In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 'TC_AUC',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True

In [2]:
op_train = pd.read_csv('../data/operation_train_new.csv')
trans_train = pd.read_csv('../data/transaction_train_new.csv')

op_test = pd.read_csv('../data/operation_round1_new.csv')
trans_test = pd.read_csv('../data/transaction_round1_new.csv')
y = pd.read_csv('../data/tag_train_new.csv')
sub = pd.read_csv('../data/submit_sample.csv')

/home/alexliu/software/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
y = pd.read_csv('../data/tag_train_new.csv')

In [3]:
def get_feature(op,trans,label):
    for feature in op.columns[2:]:
        label = label.merge(op.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
        label =label.merge(op.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
    
    for feature in trans.columns[2:]:
        if trans_train[feature].dtype == 'object':
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
        else:
            print(feature)
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].max().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].min().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].sum().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].mean().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].std().reset_index(),on='UID',how='left')
    return label

In [4]:
train = get_feature(op_train,trans_train,y)
test = get_feature(op_test,trans_test,sub)

day
trans_amt
bal
trans_type2
market_type
day
trans_amt
bal
trans_type2
market_type


In [5]:
list(train.columns)

['UID',
 'Tag',
 'mode_x',
 'mode_y',
 'success_x',
 'success_y',
 'time_x',
 'time_y',
 'os_x',
 'os_y',
 'version_x',
 'version_y',
 'device1_x',
 'device1_y',
 'device2_x',
 'device2_y',
 'device_code1_x',
 'device_code1_y',
 'device_code2_x',
 'device_code2_y',
 'device_code3_x',
 'device_code3_y',
 'mac1_x',
 'mac1_y',
 'mac2_x',
 'mac2_y',
 'ip1_x',
 'ip1_y',
 'ip2_x',
 'ip2_y',
 'wifi_x',
 'wifi_y',
 'geo_code_x',
 'geo_code_y',
 'ip1_sub_x',
 'ip1_sub_y',
 'ip2_sub_x',
 'ip2_sub_y',
 'day_x',
 'day_y',
 'day_x',
 'day_y',
 'day_x',
 'day_y',
 'day',
 'time_x',
 'time_y',
 'trans_amt_x',
 'trans_amt_y',
 'trans_amt_x',
 'trans_amt_y',
 'trans_amt_x',
 'trans_amt_y',
 'trans_amt',
 'amt_src1_x',
 'amt_src1_y',
 'merchant_x',
 'merchant_y',
 'code1_x',
 'code1_y',
 'code2_x',
 'code2_y',
 'trans_type1_x',
 'trans_type1_y',
 'acc_id1_x',
 'acc_id1_y',
 'device_code1_x',
 'device_code1_y',
 'device_code2_x',
 'device_code2_y',
 'device_code3_x',
 'device_code3_y',
 'device1_x',
 'de

In [ ]:
train.head()

In [ ]:
train = train.fillna(-1)
test = test.fillna(-1)

In [ ]:
train = train.drop(['UID','Tag'],axis = 1).fillna(-1)
label = y['Tag']

In [ ]:
test_id = test['UID']
test = test.drop(['UID','Tag'],axis = 1).fillna(-1)